In [3]:
import gzip 
import shutil 
import os 
import pandas as pd
from ast import literal_eval
import json
from datetime import datetime
from sqlalchemy import create_engine

In [29]:
#Convert files to json
with gzip.open('users.json.gz', 'rb') as f_in:
      with open('users.json.gz'.replace('.gz',''), 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [9]:
#Read json files
def parse_to_df(json_file):
    with open(json_file) as f:
        data = [json.loads(line) for line in f]
        return pd.json_normalize(data)
    
    
receipts = parse_to_df('receipts.json')
brands = parse_to_df('brands.json')
users = parse_to_df('users 2.json')

In [13]:
receipts.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12


# Incomplete Item List - Possible loss of partner information

In [45]:
receipts[(receipts['rewardsReceiptStatus'] == 'FINISHED') & (receipts['purchasedItemCount'] == 0.0)]

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
396,{'$oid': '6009eb000a7214ada2000003'},250.0,"Receipt number 3 completed, bonus point schedu...",{'$date': 1611262720000},{'$date': 1611262720000},{'$date': 1611262746000},{'$date': 1611262755000},{'$date': 1611262746000},250.0,{'$date': 1611187200000},0.0,NaN,FINISHED,0.0,6009e60450b3311194385009
424,{'$oid': '600aff160a720f053500000c'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1611333398000},{'$date': 1611333398000},{'$date': 1611333421000},{'$date': 1611333433000},{'$date': 1611333421000},500.0,{'$date': 1611273600000},0.0,NaN,FINISHED,0.0,600afb2a7d983a124e9aded0


The above two data points seem to be a data quality issue as although the points have been awarded to the user, there is no information about the items and their corresponding barcodes. If the receipts had any partner purchases, that information is lost as well and consequently so is the partner revenue.

In [60]:
receipts['rewardsReceiptStatus'].unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

In [15]:
brands.head()

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


# Duplicate user records

### About 50% records are duplicated in the users table, these records contribute to a data quality issue as not much insight can be derived from them

In [17]:
users.head()

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12


In [22]:
users.count()

active               495
role                 495
signUpSource         447
state                439
_id.$oid             495
createdDate.$date    495
lastLogin.$date      433
dtype: int64

In [20]:
users[users.duplicated()]

,active,role,signUpSource,state,_id.$oid,createdDate.$date,lastLogin.$date
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
5,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
8,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,1.609688e+12
...,...,...,...,...,...,...,...
490,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
491,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
492,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12
493,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,1418998882381,1.614963e+12


# Barcode Duplication

In [30]:
brands['barcode'].count()

1167

In [32]:
brands['barcode'].nunique()

1160

In [34]:
brands[brands['barcode'].duplicated()]

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
299,511111504139,Grocery,NaN,Pace,False,5a8c33f3e4b07f0a2dac8943,5a734034e4b0d58f376be874,Cogs,PACE
412,511111504788,Condiments & Sauces,NaN,The Pioneer Woman,NaN,5ccb2ece166eb31bbbadccbe,559c2234e4b06aca36af13c6,Cogs,PIONEER WOMAN
536,511111204923,Snacks,NaN,CHESTER'S,NaN,5d6027f46d5f3b23d1bc7906,5332f5fbe4b03c9a25efd0ba,Cogs,CHESTERS
651,511111305125,Magazines,NaN,Rachael Ray Everyday,NaN,5d642d65a3a018514994f42d,5d5d4fd16d5f3b23d1bc7905,Cogs,511111305125
1012,511111605058,Dairy,NaN,Brand2,True,5c4637ba87ff35681e840d57,5c45f8b087ff3552f950f026,Cogs,09090909090
1015,511111704140,NaN,NaN,Diet Chris Cola,NaN,5a7e0665e4b0aedb3b84afd4,55b62995e4b0d8e685c14213,Cogs,DIETCHRIS2
1071,511111004790,Condiments & Sauces,NaN,Bitten Dressing,NaN,5cdacd63166eb33eb7ce0fa8,559c2234e4b06aca36af13c6,Cogs,BITTEN


In [36]:
brands[brands['barcode'] == '511111004790']

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
467,511111004790,Baking,NaN,alexa,True,5c409ab4cd244a3539b84162,55b62995e4b0d8e685c14213,Cogs,ALEXA
1071,511111004790,Condiments & Sauces,NaN,Bitten Dressing,NaN,5cdacd63166eb33eb7ce0fa8,559c2234e4b06aca36af13c6,Cogs,BITTEN


There are duplicate barcodes found in the brands dataframe which pose a data quality issue. (Assumption: that one barcode should universally map to only one item or product even though the partner ids are different)

# CategoryCode missing values

### About 50% of values in the categoryCode column is missing

In [66]:
brands['categoryCode'].isna().sum()

650

In [62]:
brands['categoryCode'].count()

517

In [68]:
brands['categoryCode']

0                 BAKING
1              BEVERAGES
2                 BAKING
3                 BAKING
4       CANDY_AND_SWEETS
              ...       
1162              BAKING
1163                 NaN
1164    CANDY_AND_SWEETS
1165                 NaN
1166              BAKING
Name: categoryCode, Length: 1167, dtype: object

# BrandCode values

##### Values in brandcode aren't homogenous, a mix of numbers only and alphanumeric letters found

In [76]:
brands['brandCode'].unique()

array([nan, 'STARBUCKS', 'TEST BRANDCODE @1612366146176',
       'TEST BRANDCODE @1612366146051', 'TEST BRANDCODE @1612366146827',
       'TEST BRANDCODE @1612366146091', 'TEST BRANDCODE @1612366146133',
       'J.L. KRAFT', 'CAMPBELLS HOME STYLE', 'TEST',
       'TEST BRANDCODE @1598813526777', 'CALUMET', '511111205012',
       'AUNT JEMIMA SYRUP', 'MOLSON', 'LOTRIMIN',
       'TEST BRANDCODE @1597342520277', 'ST IVES', 'CHRISIMAGE',
       'ALKA SELTZER', "JACK DANIEL'S BARBECUE", 'MAGNUM Ice Cream',
       '511111105329', 'TEST BRANDCODE @1598635634882', 'TACO BELL',
       'FROSTED CHEERIOS', 'TEST BRANDCODE @1598639199674',
       'GODIVA DRY PACKAGED DESSERTS', 'LARABAR',
       'TEST BRANDCODE @1597350074333', 'TEST BRANDCODE @1607636368717',
       'TEST BRANDCODE @1607707830095', 'COTTONELLE', 'IZZE', 'MIO',
       '511111505365', 'QUILTING SPECIAL EDITION',
       'TEST BRANDCODE @1604437351617', 'HERMAN', 'KEVITA', 'DELIMEX',
       'THE RIGHT TO SHOWER', 'CARESS', 'TEST BRA